In [12]:
from env import ImgRouterEvalEnv
from lightning_lite.utilities.seed import seed_everything
from bandit_algo import BanditAlgorithm
from preference_estimator import (
    EmpiricalEstimator,
    NeuralEstimator,
    RecurrentNeuralEstimator,
    RecurrentNeuralEstimatorV0,
)
from exploration_strategy import (
    PerArmExploration,
    UniformEpsilonExploration,
)
import numpy as np

In [8]:
num_candidates = 64
cfg = {
    "num_candidates": num_candidates,
}
env = ImgRouterEvalEnv(cfg)

In [3]:
seed_everything(0)

obs = env.reset()
obs

Global seed set to 0


Num parameters: 14648


array([[ 1, 19],
       [ 1, 13],
       [ 1, 19],
       [ 1, 15],
       [ 0,  7],
       [ 1, 16],
       [ 1, 14],
       [ 0,  8],
       [ 1, 14],
       [ 0,  4]])

In [5]:
batch_x, batch_z, batch_y = env._get_full_obs()
batch_y

tensor([19, 13, 19, 15,  7, 16, 14,  8, 14,  4])

In [11]:
num_tasks = 2
num_cls = 10
reduce_fator = 2
num_slates = num_candidates // reduce_fator
estimator = EmpiricalEstimator(num_tasks, num_cls)
# estimator = RecurrentNeuralEstimatorV0(num_tasks, num_cls)
# explore = PerArmExploration(num_tasks, num_cls, num_slates)
explore = UniformEpsilonExploration(num_tasks, num_cls, num_slates)
algo = BanditAlgorithm(
        estimator,
        explore,
    )

In [13]:
obs = env.reset()
# print("initial obs:", obs)
done = False
step_rewards = []
pred_losses = []
step = 0
while not done:
    action = algo.predict(obs)
    next_obs, reward, done, info = env.step(action)
    algo.update_estimator(obs, action, info["rewards"])
    obs = next_obs
    step_rewards.append(reward)
    pred_loss = (algo.estimator.Q - env.Q) ** 2
    pred_losses.append(pred_loss)
    step += 1

print("current samples after training:")
print(env.current_samples)
# routed_data = env.current_samples - existing_samples
routed_data = env.current_samples
print("routed data:")
print(routed_data)
print(np.mean(step_rewards))

Num parameters: 14648


IndexError: index 16 is out of bounds for axis 1 with size 10